In [1]:
from openbabel import pybel
import numpy as np
import torch
import torch.nn as nn
import torchani
from torchani.repulsion import RepulsionCalculator, StandaloneRepulsionCalculator
from torchani.units import hartree2kcalmol 

device = torch.device("cuda:0")

/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/torchani-2.3.dev568+geffb1bb-py3.7.egg/torchani/infer.py:16: UserWarning: mnp not installed
  warnings.warn("mnp not installed")
/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/torchani-2.3.dev568+geffb1bb-py3.7.egg/torchani/aev/aev_computer.py:24: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [2]:
def mols2arrays(sdf_fil):
    species_order = ("H", 'C', 'N', 'O', 'S', 'F', 'Cl')
    ani2xt_index = {1:0, 6:1, 7:2, 8:3, 16:4, 9:5, 17:6}
    coord = np.array([[a.coords for a in mol.atoms] for mol in mols])
    numbers = np.array([[ani2xt_index[a.atomicnum] for a in mol.atoms] for mol in mols])
    return coord, numbers

class ANI2xt(nn.Module):
    def __init__(self, device, state_dict='models/ani2xt.pt'):
        super().__init__()
        self.device = device
        self.state_dict = state_dict
        # setup constants and construct an AEV computer
        Rcr = 5.2000e+00
        Rca = 3.5000e+00
        EtaR = torch.tensor([1.6000000e+01], device=device)
        ShfR = torch.tensor([9.0000000e-01, 1.1687500e+00, 1.4375000e+00, 1.7062500e+00, 1.9750000e+00, 2.2437500e+00, 2.5125000e+00, 2.7812500e+00, 3.0500000e+00, 3.3187500e+00, 3.5875000e+00, 3.8562500e+00, 4.1250000e+00, 4.3937500e+00, 4.6625000e+00, 4.9312500e+00], device=device)
        Zeta = torch.tensor([3.2000000e+01], device=device)
        ShfZ = torch.tensor([1.9634954e-01, 5.8904862e-01, 9.8174770e-01, 1.3744468e+00, 1.7671459e+00, 2.1598449e+00, 2.5525440e+00, 2.9452431e+00], device=device)
        EtaA = torch.tensor([8.0000000e+00], device=device)
        ShfA = torch.tensor([9.0000000e-01, 1.5500000e+00, 2.2000000e+00, 2.8500000e+00], device=device)
#         species_order = [b'H', b'C', b'N', b'O', b'S', b'F', b'Cl']
        species_order = ["H", 'C', 'N', 'O', 'S', 'F', 'Cl']

        num_species = len(species_order)
        aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, ShfR, EtaA, Zeta, ShfA, ShfZ, num_species)
        aev_dim = aev_computer.aev_length

        H_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 256),
            torch.nn.CELU(0.1),
            torch.nn.Linear(256, 192),
            torch.nn.CELU(0.1),
            torch.nn.Linear(192, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 1)
        )

        C_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 224),
            torch.nn.CELU(0.1),
            torch.nn.Linear(224, 192),
            torch.nn.CELU(0.1),
            torch.nn.Linear(192, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 1)
        )

        N_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 192),
            torch.nn.CELU(0.1),
            torch.nn.Linear(192, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 128),
            torch.nn.CELU(0.1),
            torch.nn.Linear(128, 1)
        )

        O_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 192),
            torch.nn.CELU(0.1),
            torch.nn.Linear(192, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 128),
            torch.nn.CELU(0.1),
            torch.nn.Linear(128, 1)
        )

        S_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 128),
            torch.nn.CELU(0.1),
            torch.nn.Linear(128, 96),
            torch.nn.CELU(0.1),
            torch.nn.Linear(96, 1)
        )

        F_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 128),
            torch.nn.CELU(0.1),
            torch.nn.Linear(128, 96),
            torch.nn.CELU(0.1),
            torch.nn.Linear(96, 1)
        )

        Cl_network = torch.nn.Sequential(
            torch.nn.Linear(aev_dim, 160),
            torch.nn.CELU(0.1),
            torch.nn.Linear(160, 128),
            torch.nn.CELU(0.1),
            torch.nn.Linear(128, 96),
            torch.nn.CELU(0.1),
            torch.nn.Linear(96, 1)
        )

        nn = torchani.ANIModel([H_network, C_network, N_network, O_network, S_network, F_network, Cl_network])
        checkpoint = torch.load(self.state_dict, map_location=self.device)
        nn.load_state_dict(checkpoint)
        self.shifter = torchani.utils.EnergyShifter(torch.tensor([ -0.6002,  -38.1224,  -54.7241,  -75.2086, 
                                                                    -398.1251,  -99.8006, -460.1375], 
                                                                    device=self.device, dtype=torch.float64))
        self.model = torchani.nn.Sequential(aev_computer, nn).to(device)
        
#         self.species_order = ("H", 'C', 'N', 'O', 'S', 'F', 'Cl')
        self.rep = StandaloneRepulsionCalculator(elements=species_order, periodic_table_index=False).to(device)
#         self.model.train(False)

    
    def forward(self, species, coords):
        #Electronic part
        energy = self.shifter(self.model((species, coords))).energies
        gradient = torch.autograd.grad([energy.sum()], [coords])[0]
        force = -gradient
    
        #Repulsion part
        rep_energy = self.rep((species, coords)).energies
        derivative = torch.autograd.grad(rep_energy.sum(), coords)[0]
        rep_force = -derivative
        
        final_energy = energy + rep_energy
        final_force = force + rep_force
        return (final_energy, final_force)



In [3]:
# Repulsion energy and forces
# species_order = ("H", 'C', 'N', 'O', 'S', 'F', 'Cl')
# rep = StandaloneRepulsionCalculator(elements=species_order, periodic_table_index=False).to(device)

f = "/home/liucmu/auto3D/workflow/batch_opt/test100.sdf"
f2 = "/home/liucmu/auto3D/workflow/batch_opt/test2.sdf"
mols = list(pybel.readfile('sdf', f2))
coord, numbers = mols2arrays(mols)
print(coord.shape, numbers.shape)

coord = torch.tensor(coord, dtype=torch.float64, requires_grad=True, device=device).float()
species = torch.tensor(numbers, dtype=torch.long, device=device)

(2,) (2,)


/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.
/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [4]:
# ANI2xt energy and forces
ani2xt = ANI2xt(device, state_dict='/home/liucmu/auto3D/workflow/models/ani2xt.pt')

/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/torchani-2.3.dev568+geffb1bb-py3.7.egg/torchani/utils.py:294: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self_energies = torch.tensor(self_energies, dtype=torch.double)


In [5]:
print(ani2xt)

ANI2xt(
  (shifter): EnergyShifter()
  (model): Sequential(
    (0): AEVComputer(
      (angular_terms): StandardAngular(
        (cutoff_fn): CutoffCosine()
      )
      (radial_terms): StandardRadial(
        (cutoff_fn): CutoffCosine()
      )
      (neighborlist): FullPairwise()
    )
    (1): ANIModel(
      (0): Sequential(
        (0): Linear(in_features=1008, out_features=256, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=256, out_features=192, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=192, out_features=160, bias=True)
        (5): CELU(alpha=0.1)
        (6): Linear(in_features=160, out_features=1, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=1008, out_features=224, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=224, out_features=192, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=192, out_features=160, bias=True)
        (5): CELU(alpha=0.1)


In [6]:
print(species.shape, coord.shape)
e, f = ani2xt(species, coord)
print(e.size(), f.size())

torch.Size([100, 23]) torch.Size([100, 23, 3])
torch.Size([100]) torch.Size([100, 23, 3])


In [7]:
print(e)

tensor([-612.4149, -612.4152, -612.4145, -612.4130, -612.4192, -612.4322,
        -612.4315, -612.4190, -612.4199, -612.4145, -612.4167, -612.4210,
        -612.4157, -612.4252, -612.4227, -612.4123, -612.4212, -612.4182,
        -612.4103, -612.4162, -612.4165, -612.4093, -612.4110, -612.4005,
        -612.4043, -612.4265, -612.4108, -612.4109, -612.4178, -612.4153,
        -612.4110, -612.4025, -612.4054, -612.4193, -612.4091, -612.4187,
        -612.4244, -612.4065, -612.4345, -612.4116, -612.4200, -612.4200,
        -612.4094, -612.4120, -612.3883, -612.4274, -612.4202, -612.4151,
        -612.4162, -612.4085, -612.4070, -612.4196, -612.4221, -612.3979,
        -612.4320, -612.4278, -612.4259, -612.4216, -612.4101, -612.4235,
        -612.4080, -612.4258, -612.4057, -612.4034, -612.4145, -612.4074,
        -612.4117, -612.4316, -612.4200, -612.4236, -612.4276, -612.4061,
        -612.4100, -612.4095, -612.4084, -612.4228, -612.4117, -612.4105,
        -612.4178, -612.4219, -612.419

In [8]:
print(f)

tensor([[[ 0.0874,  0.0021,  0.0089],
         [-0.0821, -0.1736,  0.0256],
         [-0.0326, -0.0429,  0.0072],
         ...,
         [ 0.0127,  0.0706, -0.0476],
         [-0.0626, -0.0512, -0.0202],
         [-0.0546,  0.0329,  0.0982]],

        [[-0.0811,  0.0759, -0.0304],
         [-0.0949, -0.1333,  0.0583],
         [ 0.1338, -0.0816,  0.0456],
         ...,
         [-0.0902, -0.0747, -0.0263],
         [ 0.0138,  0.0724, -0.0881],
         [-0.0367,  0.0611,  0.0664]],

        [[ 0.1318,  0.0148,  0.0071],
         [-0.1010, -0.1883,  0.0903],
         [-0.0427, -0.0106,  0.0014],
         ...,
         [-0.0477,  0.0214,  0.0535],
         [ 0.0128,  0.0891, -0.0776],
         [-0.0583, -0.0734, -0.0442]],

        ...,

        [[-0.0802, -0.0682, -0.0255],
         [-0.1031,  0.1351,  0.0745],
         [ 0.1780,  0.0580,  0.0304],
         ...,
         [-0.0174, -0.0627, -0.0996],
         [-0.0277, -0.0399,  0.0647],
         [-0.0729,  0.0808, -0.0146]],

        [[

# Ani2xt on different molecules

In [1]:
from openbabel import pybel
import numpy as np
import torch
import torch.nn as nn
import torchani
from torchani.repulsion import RepulsionCalculator, StandaloneRepulsionCalculator
from torchani.units import hartree2kcalmol
from utils import *

device = torch.device("cuda:0")

/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/torchani-2.3.dev568+geffb1bb-py3.7.egg/torchani/infer.py:16: UserWarning: mnp not installed
  warnings.warn("mnp not installed")
/home/liucmu/miniconda3/envs/py37/lib/python3.7/site-packages/torchani-2.3.dev568+geffb1bb-py3.7.egg/torchani/aev/aev_computer.py:24: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [4]:
# Repulsion energy and forces
# species_order = ("H", 'C', 'N', 'O', 'S', 'F', 'Cl')
# rep = StandaloneRepulsionCalculator(elements=species_order, periodic_table_index=False).to(device)

f = "/home/liucmu/auto3D/workflow/batch_opt/test100.sdf"
f2 = "/home/liucmu/auto3D/workflow/batch_opt/test2.sdf"
mols = list(pybel.readfile('sdf', f2))
coord, numbers = mols2lists(mols)

coord_padded = padding_coords(coord[1:])
numbers_padded = padding_species(numbers[1:])
print(coord_padded, numbers_padded)

# coord = torch.tensor(coord, dtype=torch.float64, requires_grad=True, device=device).float()
# species = torch.tensor(numbers, dtype=torch.long, device=device)

[[(3.5291, 0.1498, 0.2911), (2.752, -0.9304, 0.6543), (1.3866, -0.9348, 0.4238), (0.7977, 0.1598, -0.1788), (-0.6492, 0.1758, -0.4363), (-1.159, 1.1824, -0.979), (-1.5111, -0.9655, -0.0709), (-2.8731, -0.6811, -0.4321), (-3.4636, 0.3714, 0.3598), (1.5829, 1.2328, -0.5372), (2.9427, 1.2552, -0.3158), (4.589, 0.0881, 0.4994), (3.196, -1.8143, 1.1359), (0.7444, -1.7585, 0.6924), (-1.1934, -1.8247, -0.7339), (-1.4173, -1.2787, 0.9786), (-2.951, -0.5103, -1.457), (-2.804, 0.8004, 1.1135), (-3.8451, 1.1851, -0.2935), (-4.336, -0.0946, 0.8937), (1.1195, 2.0844, -1.0067), (3.5627, 2.1078, -0.6015)]] [[1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [5]:
coords = torch.tensor(coord_padded, dtype=torch.float64, requires_grad=True, device=device).float()
species = torch.tensor(numbers_padded, dtype=torch.long, device=device)

ani2xt = ANI2xt(device, state_dict='/home/liucmu/auto3D/workflow/models/ani2xt.pt')

print(species.shape, coords.shape)
e, f = ani2xt(species, coords)
print(e.size(), f.size())
print(e, f)

torch.Size([1, 22]) torch.Size([1, 22, 3])
torch.Size([1]) torch.Size([1, 22, 3])
tensor([-479.2707], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>) tensor([[[ 0.0936,  0.0158,  0.0086],
         [ 0.0319, -0.1198,  0.0603],
         [-0.0293, -0.0727,  0.0289],
         [-0.1148, -0.0251, -0.0072],
         [ 0.0396,  0.0324,  0.0035],
         [-0.0553,  0.1102, -0.0629],
         [-0.0458, -0.0487, -0.0639],
         [-0.0360, -0.0847,  0.0275],
         [-0.0630, -0.0320, -0.0316],
         [-0.0255,  0.1054, -0.0522],
         [ 0.0796,  0.0623, -0.0150],
         [ 0.1309,  0.0017,  0.0215],
         [ 0.0437, -0.0735,  0.0412],
         [-0.0687, -0.1123,  0.0389],
         [ 0.0232, -0.0503, -0.0206],
         [ 0.0106, -0.0283,  0.1032],
         [-0.0035,  0.0213, -0.0652],
         [ 0.0779,  0.0474,  0.0817],
         [-0.0279,  0.0646, -0.0426],
         [-0.0541, -0.0123,  0.0387],
         [-0.0639,  0.1116, -0.0625],
         [ 0.0568,  0.0871, -0.

In [3]:
coord[1]

[(3.5291, 0.1498, 0.2911),
 (2.752, -0.9304, 0.6543),
 (1.3866, -0.9348, 0.4238),
 (0.7977, 0.1598, -0.1788),
 (-0.6492, 0.1758, -0.4363),
 (-1.159, 1.1824, -0.979),
 (-1.5111, -0.9655, -0.0709),
 (-2.8731, -0.6811, -0.4321),
 (-3.4636, 0.3714, 0.3598),
 (1.5829, 1.2328, -0.5372),
 (2.9427, 1.2552, -0.3158),
 (4.589, 0.0881, 0.4994),
 (3.196, -1.8143, 1.1359),
 (0.7444, -1.7585, 0.6924),
 (-1.1934, -1.8247, -0.7339),
 (-1.4173, -1.2787, 0.9786),
 (-2.951, -0.5103, -1.457),
 (-2.804, 0.8004, 1.1135),
 (-3.8451, 1.1851, -0.2935),
 (-4.336, -0.0946, 0.8937),
 (1.1195, 2.0844, -1.0067),
 (3.5627, 2.1078, -0.6015)]